In [3]:
import numpy as np
from numpy.testing import assert_allclose 
import keras
from keras.models import Model
from keras import regularizers
from keras.models import load_model
from keras.layers import Dense, Input, Dropout, LSTM, Activation, RNN, Bidirectional
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.initializers import glorot_uniform
from keras.callbacks.callbacks import ModelCheckpoint
import pandas as pd
#from emo_utils import *

In [4]:
df=pd.read_csv("/home/atswi/Downloads/imdb_master1.csv",dtype=str)
cols=[4,5,6,7,8,9,10]
df.drop(df.columns[cols],axis=1,inplace=True)

max_len = 250
#len(max(df['review'], key=len).split())
df_train1=df.loc[df['type']=='train']
df_train=df_train1.loc[df['label']!='unsup']
df_train_pos=df_train.loc[df['label']=='pos']
df_train_neg=df_train.loc[df['label']=='neg']
frames=[df_train_pos,df_train_neg]
df_train=pd.concat(frames)
df_train = df_train.sample(frac=1).reset_index(drop=True)
df_test=df.loc[df['type']=='test']
df_test_pos=df_test.loc[df['label']=='pos']
df_test_neg=df_test.loc[df['label']=='neg']
frames=[df_test_pos,df_test_neg]
df_test=pd.concat(frames)
df_test = df_test.sample(frac=1).reset_index(drop=True)

X_train=df_train['review']
Y_train1=df_train['label']
Y_train1=Y_train1.replace(("neg","pos"),(0,1))
X_test=df_test['review']
Y_test1=df_test['label']
Y_test1=Y_test1.replace(("neg","pos"),(0,1))
Y_train=Y_train1.to_numpy()
print(Y_train)
Y_test=Y_test1.to_numpy()
X_train=np.asarray(X_train)
X_test=np.asarray(X_test)
#X_train=X_train[0:4000]
#Y_train=Y_train[0:4000]

[0 1 0 ... 1 1 0]


In [4]:
word_to_index, index_to_word, word_to_vec_map=read_glove_vecs("/home/atswi/Documents/glove.6B.100d.txt")

In [5]:
def string_to_indices(X,word_to_index,max_len):

 m=len(X)

 X_indices=np.zeros((m,max_len))

 for i in range(m):

   string_words = [p.lower() for p in X[i].split()]

   j=0

   for w in string_words:
    try:
       X_indices[i,j]=word_to_index[w]
       j=j+1
       if(j==max_len):
        break
    except: KeyError
 return X_indices


def pretrained_embedding_layer(word_to_vec_map,word_to_index):
    vocab_len = len(word_to_index)+1
    emb_dim = 100

    emb_matrix=np.zeros((vocab_len,emb_dim))

    for word,idx in word_to_index.items():
        emb_matrix[idx, :]=word_to_vec_map[word]

    embedding_layer = Embedding(vocab_len,emb_dim)
    #embeddings_initializer='glorot_normal'

    embedding_layer.build((None,))

    embedding_layer.set_weights([emb_matrix])

    return embedding_layer


In [6]:
print(word_to_vec_map['the'])

[-0.038194 -0.24487   0.72812  -0.39961   0.083172  0.043953 -0.39141
  0.3344   -0.57545   0.087459  0.28787  -0.06731   0.30906  -0.26384
 -0.13231  -0.20757   0.33395  -0.33848  -0.31743  -0.48336   0.1464
 -0.37304   0.34577   0.052041  0.44946  -0.46971   0.02628  -0.54155
 -0.15518  -0.14107  -0.039722  0.28277   0.14393   0.23464  -0.31021
  0.086173  0.20397   0.52624   0.17164  -0.082378 -0.71787  -0.41531
  0.20335  -0.12763   0.41367   0.55187   0.57908  -0.33477  -0.36559
 -0.54857  -0.062892  0.26584   0.30205   0.99775  -0.80481  -3.0243
  0.01254  -0.36942   2.2167    0.72201  -0.24978   0.92136   0.034514
  0.46745   1.1079   -0.19358  -0.074575  0.23353  -0.052062 -0.22044
  0.057162 -0.15806  -0.30798  -0.41625   0.37972   0.15006  -0.53212
 -0.2055   -1.2526    0.071624  0.70565   0.49744  -0.42063   0.26148
 -1.538    -0.30223  -0.073438 -0.28312   0.37104  -0.25217   0.016215
 -0.017099 -0.38984   0.87424  -0.72569  -0.51058  -0.52028  -0.1459
  0.8278    0.27062 ]

In [13]:
def SentiAnalysis(input_shape, word_to_vec_map, word_to_index):

    string_indices= Input(shape=input_shape,dtype='int32')
    embedding_layer=pretrained_embedding_layer(word_to_vec_map,word_to_index)
    embeddings=embedding_layer(string_indices)
    X= Bidirectional(LSTM(units=32,return_sequences=True))(embeddings)
    X= Dropout(rate=0.7)(X)
    X= Bidirectional(LSTM(units=16,return_sequences=False))(X)
    X= Dropout(rate=0.5)(X)
    X= Dense(units=1)(X)
    X=Activation('sigmoid')(X)

    model= Model(inputs=string_indices,outputs=X)

    return model

In [18]:
filepath = "model.h5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [20]:
model = SentiAnalysis((max_len,),word_to_vec_map,word_to_index)

#keras.optimizers.Nadam(learning_rate=0.1, beta_1=0.9, beta_2=0.999)
model.compile(loss='binary_crossentropy',optimizer=keras.optimizers.SGD(learning_rate=0.03, momentum=0.95, nesterov=False),metrics=['accuracy'])

X_train_indices = string_to_indices(X_train,word_to_index,max_len)

model.fit(X_train_indices,Y_train,epochs=20,batch_size=100,shuffle=True,callbacks=callbacks_list)

/home/atswi/.local/lib/python3.6/site-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/20
24973/24973 [==============================] - 231s 9ms/step - loss: 0.6862 - accuracy: 0.5484

Epoch 00001: loss did not improve from 0.67786
Epoch 2/20
24973/24973 [==============================] - 228s 9ms/step - loss: 0.6130 - accuracy: 0.6756

Epoch 00002: loss improved from 0.67786 to 0.61305, saving model to model.h5
Epoch 3/20
24973/24973 [==============================] - 226s 9ms/step - loss: 0.4773 - accuracy: 0.7917

Epoch 00003: loss improved from 0.61305 to 0.47730, saving model to model.h5
Epoch 4/20
24973/24973 [==============================] - 218s 9ms/step - loss: 0.4271 - accuracy: 0.8181

Epoch 00004: loss improved from 0.47730 to 0.42713, saving model to model.h5
Epoch 5/20
24973/24973 [==============================] - 230s 9ms/step - loss: 0.3966 - accuracy: 0.8344

Epoch 00005: loss improved from 0.42713 to 0.39661, saving model to model.h5
Epoch 6/20
24973/24973 [==============================] - 229s 9ms/step - loss: 0.3747 - accuracy: 0.8470

Epo

In [21]:
X_test_indices = string_to_indices(X_test,word_to_index,max_len)
loss,acc=model.evaluate(X_test_indices,Y_test)

print("Loss=",loss)
print("Accuracy=",acc)

24982/24982 [==============================] - 101s 4ms/step
Loss= 0.39607206166635167
Accuracy= 0.8545352816581726


In [28]:
new_model = load_model("model.h5")
#assert_allclose(model.predict(X_train),new_model.predict(X_train),1e-5)

/home/atswi/.local/lib/python3.6/site-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


In [29]:
filepath = "model.h5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [30]:
model.compile(loss='binary_crossentropy',optimizer=keras.optimizers.SGD(learning_rate=0.035, momentum=0.8, nesterov=False),metrics=['accuracy'])
model.fit(X_train_indices,Y_train,epochs=10,batch_size=100,shuffle=True,callbacks=callbacks_list)

/home/atswi/.local/lib/python3.6/site-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/10
24973/24973 [==============================] - 257s 10ms/step - loss: 0.1715 - accuracy: 0.9449

Epoch 00001: loss improved from inf to 0.17153, saving model to model.h5
Epoch 2/10
24973/24973 [==============================] - 247s 10ms/step - loss: 0.1621 - accuracy: 0.9494

Epoch 00002: loss improved from 0.17153 to 0.16207, saving model to model.h5
Epoch 3/10
24973/24973 [==============================] - 251s 10ms/step - loss: 0.1534 - accuracy: 0.9533

Epoch 00003: loss improved from 0.16207 to 0.15338, saving model to model.h5
Epoch 4/10
24973/24973 [==============================] - 251s 10ms/step - loss: 0.1512 - accuracy: 0.9533

Epoch 00004: loss improved from 0.15338 to 0.15116, saving model to model.h5
Epoch 5/10
24973/24973 [==============================] - 249s 10ms/step - loss: 0.1450 - accuracy: 0.9576

Epoch 00005: loss improved from 0.15116 to 0.14498, saving model to model.h5
Epoch 6/10
24973/24973 [==============================] - 247s 10ms/step - loss

In [31]:
loss,acc=model.evaluate(X_test_indices,Y_test)

print("Loss=",loss)
print("Accuracy=",acc)

24982/24982 [==============================] - 102s 4ms/step
Loss= 0.5580819603626841
Accuracy= 0.8486510515213013
